# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [1]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

from math import sqrt

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [2]:
def solve_tsp_nearest_neighbour(instance):
    n = len(instance)
    perm = [0]
    used = [False] * n
    used[0] = True
    last = 0
    
    for it in range(n - 1):
        best = -1
        for i in range(n):
            if (used[i] == True):
                continue
            
            if (best == -1 or (euclidean_distance(instance[last], instance[i]) < euclidean_distance(instance[last], instance[best]))):
                best = i
                
        used[i] = True
        perm.append(i)
        last = i
    return perm

In [3]:
def solve_tsp_nearest_insertion(instance):
    n = len(instance)
    perm = [0]
    used = [False] * n
    used[0] = True
    
    for it in range(n - 2):
        best = -1
        best_dst = 1e18
        
        for i in range(n):
            if (used[i]):
                continue
            
            dst = 1e18
            
            for j in range(len(perm)):
                dst = min(dst, euclidean_distance(instance[i], instance[perm[j]]))
                
            if (dst < best_dst):
                best = i
                best_dst = dst
        
        if (len(perm) == 1):
            perm.append(best)
            continue
        
        pos = 0;
        best_dst = 1e18
        
        for i in range(len(perm)):
            now_dst = euclidean_distance(instance[perm[i]], instance[best])
            now_dst += euclidean_distance(instance[perm[(i + 1) % len(perm)]], instance[best])
            now_dst -= euclidean_distance(instance[perm[i]], instance[perm[(i + 1) % len(perm)]])
            
            if (now_dst < best_dst):
                best_dst = now_dst
                pos = i
                
        perm = perm[0:(pos + 1)] + [best] + perm[(pos + 1):len(perm)]
    return perm

In [4]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['d198.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done in {:.2} seconds with tour length {} using NN and in {:.2} seconds with tour length {} using NI'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [5]:
run_all()

Solving instance d198.tsp… done in 0.037 seconds with tour length 8158 using NN and in 2.1 seconds with tour length 2277 using NI
Solving instance d493.tsp… done in 0.19 seconds with tour length 7764 using NN and in 3e+01 seconds with tour length 3828 using NI
Solving instance d657.tsp… done in 0.33 seconds with tour length 8875 using NN and in 7.5e+01 seconds with tour length 2633 using NI
Solving instance d2103.tsp… done in 4.1 seconds with tour length 10408 using NN and in 2.6e+03 seconds with tour length 2201 using NI
Solving instance pr107.tsp… done in 0.0084 seconds with tour length 20771 using NN and in 0.33 seconds with tour length 400 using NI
Solving instance pr152.tsp… done in 0.018 seconds with tour length 27438 using NN and in 0.99 seconds with tour length 1177 using NI
Solving instance pr439.tsp… done in 0.17 seconds with tour length 13968 using NN and in 2.3e+01 seconds with tour length 360 using NI


## Выводы
Как мы знаем второе решение хуже первого ассимптотически, но по качеству на файлах типа d работает примерно в 4-5 раз лучше, а на файлах типа pr работает лучше примерно в 30 раз.
Эти тесты показывают нам что второе решение заведомо лучше первого, и есть тесты на которых ответ отличается во много раз.